# Processing ICON Data - 
- Reference - [ICON Google Colab](https://colab.research.google.com/github/YJWu-SSL/ICON_Data_Demo/blob/master/ICON_data_demo_Colab_v04.ipynb#scrollTo=LuavL8u8PXTh)

In [2]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter, MinuteLocator
import matplotlib.gridspec as gridspec
from matplotlib.ticker import MaxNLocator
from datetime import datetime, timedelta
import bisect
import pandas as pd
import seaborn as sns
import pylab
import os
import glob
from ftplib import FTP
import fnmatch
import matplotlib.dates as mdates
import xarray as xr

'''Optional: Specify the plotting style '''
#plt.style.use('seaborn')
plt.rcParams.update({'font.size': 15,\
                     'xtick.labelsize' : 15,\
                     'ytick.labelsize' : 15,\
                     'axes.titlesize' : 16,\
                     'axes.labelsize' : 16,\
                     'date.autoformatter.minute': '%H:%M' })

In [5]:
'''Insert a target day'''
target_datetime=datetime(2021,11,1) # (year, month, day)
target_YYYY='%4i' % target_datetime.year
target_mm='%02i' % target_datetime.month
target_dd='%02i' % target_datetime.day
target_doy='%03i' % ((target_datetime-datetime(int(target_YYYY),1,1)).days+1)
target_date_str='%s-%s-%s' % (target_YYYY,target_mm,target_dd)

In [6]:
# Load Data from file

In [7]:
'''MIGHTI L2.2 Vector Wind'''
MT_color= 'Green'
fn_L22=glob.glob('/content/%s/ICON_L2-2_*%s*.NC' % (target_doy,MT_color))[0]

dm_g=xr.open_dataset(fn_L22)
dm_g=dm_g.rename({'Epoch': 'time_ms',
                 'ICON_L22_Orbit_Number':'orb_num',
                 'ICON_L22_Longitude':'tang_lon',
                 'ICON_L22_Latitude':'tang_lat',
                 'ICON_L22_Altitude':'tang_alt',
                 'ICON_L22_Meridional_Wind': 'umer',
                 'ICON_L22_Zonal_Wind':'uzon',
                 'ICON_L22_Local_Solar_Time':'tang_slt',
                 'ICON_L22_Wind_Quality':'wind_quality'})

dm_g['time']=pd.to_datetime(np.array([datetime(1970,1,1) + timedelta(seconds = 1e-3*s) for s in dm_g['time_ms'].values]))
# Only allow the good data: dm_g.wind_quality >= 0.5
good_data=(dm_g.wind_quality == 0.5)|(dm_g.wind_quality == 1)
dm_g[['umer','uzon']]=dm_g[['umer','uzon']].where(good_data)


print('Orbit numbers: %04i to %04i' % (np.nanmin(dm_g.orb_num),np.nanmax(dm_g.orb_num)))

IndexError: list index out of range

In [ ]:
# Check out more information in the NC file
dm_g

In [ ]:
'''MIGHTI L2.2 Vector Wind'''
MT_color= 'Red'
fn_L22=glob.glob('/content/%s/ICON_L2-2_*%s*.NC' % (target_doy,MT_color))[0]
### Load winds

dm_r=xr.open_dataset(fn_L22)
dm_r=dm_r.rename({'Epoch': 'time_ms',
                 'ICON_L22_Orbit_Number':'orb_num',
                 'ICON_L22_Longitude':'tang_lon',
                 'ICON_L22_Latitude':'tang_lat',
                 'ICON_L22_Altitude':'tang_alt',
                 'ICON_L22_Meridional_Wind': 'umer',
                 'ICON_L22_Zonal_Wind':'uzon',
                 'ICON_L22_Local_Solar_Time':'tang_slt',
                 'ICON_L22_Wind_Quality':'wind_quality'})

good_data = (dm_r.wind_quality == 1)| (dm_r.wind_quality == 0.5)
dm_r['time']=pd.to_datetime(np.array([datetime(1970,1,1) + timedelta(seconds = 1e-3*s) for s in dm_r['time_ms'].values]))
dm_r[['umer','uzon']]=dm_r[['umer','uzon']].where(good_data)

print('Orbit numbers: %04i to %04i' % (np.nanmin(dm_r.orb_num),np.nanmax(dm_r.orb_num)))


In [ ]:
'''MIGHTI L2.3 Temperature'''
MTAB='A'

fn_L23=glob.glob('/content/%s/ICON_L2-3_MIGHTI-%s*.NC' % (target_doy, MTAB))[0]


dTN=xr.open_dataset(fn_L23)
dTN=dTN.rename({'Epoch': 'time_ms',
                 'ICON_L23_Orbit_Number':'orb_num',
                 'ICON_L23_MIGHTI_%s_Tangent_Longitude' % (MTAB):'tang_lon',
                 'ICON_L23_MIGHTI_%s_Tangent_Latitude' % (MTAB):'tang_lat',
                 'ICON_L23_MIGHTI_%s_Tangent_Altitude' % (MTAB):'tang_alt',
                 'ICON_L23_MIGHTI_%s_Temperature' % (MTAB): 'TN',
                 'ICON_L23_MIGHTI_%s_Tangent_Local_Solar_Time' % (MTAB):'tang_slt',
                 'ICON_L1_MIGHTI_%s_Quality_Flag_South_Atlantic_Anomaly' % (MTAB):'saa_flag',
                 'ICON_L1_MIGHTI_%s_Quality_Flag_Bad_Calibration' % (MTAB):'cali_flag'
                 })

good_data = (dTN.saa_flag == 0) & (dTN.cali_flag == 0) & (dTN.TN > 0) & (abs(dTN.TN) < 10000)
dTN['time']=pd.to_datetime(np.array([datetime(1970,1,1) + timedelta(seconds = 1e-3*s) for s in dTN['time_ms'].values]))
dTN['TN']=dTN['TN'].where(good_data)
print('Orbit numbers: %04i to %04i' % (np.nanmin(dTN.orb_num),np.nanmax(dTN.orb_num)))


### Example 1: Visualizing data in a single orbit
- This practice will demostrate the how to use orbit number and how to compare different ICON product.
- Use L2.2 Vector Wind and L2.3 Temperature for demonstration.

In [8]:
# Visualizing Single Orbit

# Select a particular orbit
target_orbit_num = 2870

dTN_select_orb_num_idx = np.argwhere(dTN['orb_num'].values== target_orbit_num)
dTN_select_idx = dTN_select_orb_num_idx[:,0]

select_dTN = {}
select_dTN['orb_num'] = dTN['orb_num'][dTN_select_idx]
select_dTN['time']    = dTN['time'][dTN_select_idx]
select_dTN['time_ms']    = dTN['time_ms'][dTN_select_idx]
select_dTN['TN']      = dTN['TN'][dTN_select_idx]
select_dTN['tang_slt']      = dTN['tang_slt'][dTN_select_idx]
select_dTN['tang_lon'] = dTN['tang_lon'][dTN_select_idx]
select_dTN['tang_lat'] = dTN['tang_lat'][dTN_select_idx]
select_dTN['tang_alt'] = dTN['tang_alt'][dTN_select_idx]


dm_select_idx=np.where((dm_g['time']>=select_dTN['time'][0]) & (dm_g['time'] < select_dTN['time'][-1]))
select_dm_g = {}
select_dm_g['orb_num'] = dm_g['orb_num'][dm_select_idx]
select_dm_g['time']    = dm_g['time'][dm_select_idx]
select_dm_g['uzon']    = dm_g['uzon'][dm_select_idx]
select_dm_g['umer']    = dm_g['umer'][dm_select_idx]
select_dm_g['tang_slt']      = dm_g['tang_slt'][dm_select_idx]
select_dm_g['tang_lon'] = dm_g['tang_lon'][dm_select_idx]
select_dm_g['tang_lat'] = dm_g['tang_lat'][dm_select_idx]
select_dm_g['tang_alt'] = dm_g['tang_alt']

dm_select_idx=np.where((dm_r['time']>=select_dTN['time'][0]) & (dm_r['time'] < select_dTN['time'][-1]))
select_dm_r = {}
select_dm_r['orb_num'] = dm_r['orb_num'][dm_select_idx]
select_dm_r['time']    = dm_r['time'][dm_select_idx]
select_dm_r['uzon']    = dm_r['uzon'][dm_select_idx]
select_dm_r['umer']    = dm_r['umer'][dm_select_idx]
select_dm_r['tang_slt']      = dm_r['tang_slt'][dm_select_idx]
select_dm_r['tang_lon'] = dm_r['tang_lon'][dm_select_idx]
select_dm_r['tang_lat'] = dm_r['tang_lat'][dm_select_idx]
select_dm_r['tang_alt'] = dm_r['tang_alt']

orbit_start_time=pd.to_datetime(select_dTN['time'][0].values).strftime("%m-%d-%y %H:%M:%S")
orbit_end_time=pd.to_datetime(select_dTN['time'][-1].values).strftime("%m-%d-%y %H:%M:%S")

print('Selected time: %s to %s' % (orbit_start_time,orbit_end_time))

NameError: name 'dTN' is not defined

### Quick view of a single orbit L2.2 Vector Wind
- Daytime observations of the red line (630.0 nm) emission span the altitude range of ~180 to 300 km while observations of the green line (557.7 nm) emission span the altitude range of 90 to ~190 km.
- Nighttime observations of the red line emission span the altitude range of ~210 to 300 km while those of the green line span 90 to ~109 km.
- Daytime observations are provided at a 30-s cadence while nighttime observations are provided at a 60-s cadence.
- How to deal with the overlapped altitudes?

In [ ]:
X_g, Y_g = np.meshgrid(select_dm_g['time'],select_dm_g['tang_alt'])
X_r, Y_r = np.meshgrid(select_dm_r['time'],select_dm_r['tang_alt'])

show_time=pd.to_datetime(select_dm_g['time'].values)

# ## Wind ###

cmap='bwr'
vmax=200
vmin=-200
variable_name='MIGHTI L2.2 Meridional Wind'
cb_label='Meridional Wind (m/s)'
xlabel='UTC'
ylabel='Altitude (km)'

plt.close()
fig,ax=plt.subplots(figsize=(16,4))
fig.suptitle('%s %s' % (variable_name, show_time[0].strftime("%m-%d-%Y")),size=20,x=0.5)
gs = gridspec.GridSpec(1,2) 

ax0=plt.subplot(gs[0,0])
im = ax0.pcolor(X_g,Y_g,select_dm_g['umer'].transpose() ,cmap=cmap,vmin=vmin,vmax=vmax)
ax0.set_ylim([90,300])
ax0.set_title('Green')
ax0.set_xlabel(xlabel)
ax0.set_ylabel(ylabel)
color_bar=fig.colorbar(im)
color_bar.set_label(cb_label,size=16)


ax0=plt.subplot(gs[0,1])
im = ax0.pcolor(X_r,Y_r,select_dm_r['umer'].transpose() ,cmap=cmap,vmin=vmin,vmax=vmax)
ax0.set_ylim([90,300])
ax0.set_title('Red')
ax0.set_xlabel(xlabel)
ax0.set_ylabel(ylabel)
color_bar=fig.colorbar(im)
color_bar.set_label(cb_label,size=16)

fig.autofmt_xdate(rotation=45)


left  = 0.1  # the left side of the subplots of the figure
right = 0.9   # the right side of the subplots of the figure
bottom = 0.05   # the bottom of the subplots of the figure
top = 0.8     # the top of the subplots of the figure
wspace = 0.2#0.2   # the amount of width reserved for blank space between subplots
hspace = 0.3#0.2   # the amount of height reserved for white space between subplots

plt.subplots_adjust(left, bottom, right, top, wspace, hspace)



### Extension: Visualizing the neutral wind and temperature at mesosphere and the lower thermosphere at a time.

In [ ]:
## Temperature ###
target_1_array=select_dTN['TN']
X_1_array, _ = np.meshgrid(pd.to_datetime(select_dTN['time'].values),select_dTN['tang_alt'][10,:])

## Green Line Wind ###
target_2_array=select_dm_g['uzon']
X_2_array, Y_2_array = np.meshgrid(select_dm_g['time'],select_dm_g['tang_alt'])

# ## Temperature ###

cmap_1='plasma'
vmax_1=300
vmin_1=150
variable_name_1='ICON L2.3-%s Temperature' % (MTAB)
xlabel_1='UTC'
ylabel_1='Altitude (km)'
cb_label_1='Temperature (K)'
tang_alt_idx=5


# ## Wind ###

cmap_2='bwr'
vmax_2=200
vmin_2=-200
variable_name_2='ICON L2.2 Zonal Wind'
xlabel_2='UTC'
ylabel_2='Altitude (km)'
cb_label_2='Zonal Wind (m/s)'

######################## PLOT #############################
show_time=pd.to_datetime(select_dTN['time'].values)
fig,ax=plt.subplots(figsize=(8,8))#fig=pp.figure(figsize=(12,10)

fig.suptitle('UTC %s to %s' % (show_time[0].strftime("%H:%M:%S"),show_time[-1].strftime("%H:%M:%S %m-%d-%y")),size=20,x=0.43)
gs = gridspec.GridSpec(3, 1, height_ratios=[1,1,1]) 

ax0=plt.subplot(gs[0,0])
im = ax0.pcolor(X_1_array.transpose(),select_dTN['tang_alt'],target_1_array ,cmap=cmap_1,vmin=vmin_1,vmax=vmax_1)#,norm=colors.Normalize(vmin=0, vmax=150))
ax0.set_ylim([90,115])
ax0.set_title(variable_name_1)
ax0.set_xlabel(xlabel_1)
ax0.set_ylabel(ylabel_1)
color_bar=fig.colorbar(im)
color_bar.set_label(cb_label_1)

ax0=plt.subplot(gs[1,0])
im = ax0.pcolor(X_2_array,Y_2_array,target_2_array.transpose() ,cmap=cmap_2,vmin=vmin_2,vmax=vmax_2)#,norm=colors.Normalize(vmin=0, vmax=150))
ax0.set_ylim([90,115])
ax0.set_title(variable_name_2)
ax0.set_xlabel(xlabel_2)
ax0.set_ylabel(ylabel_2)
color_bar=fig.colorbar(im)
color_bar.set_label(cb_label_2)

ax0=plt.subplot(gs[2,0])
im=ax0.scatter(select_dTN['tang_lon'][tang_alt_idx,:],select_dTN['tang_lat'][tang_alt_idx,:],c=select_dTN['tang_slt'][tang_alt_idx,:],cmap='twilight_shifted')
ax0.set_title('Tangent point at ~%03i km' % select_dTN['tang_alt'][tang_alt_idx,1])
ax0.set_xlabel('Longitude')
ax0.set_ylabel('Latitude')

noon_idx=np.argmin(abs(select_dTN['tang_slt'][tang_alt_idx].values-12))
noon_lon=select_dTN['tang_lon'][tang_alt_idx,noon_idx]
noon_lat=select_dTN['tang_lat'][tang_alt_idx,noon_idx]
ax0.plot(noon_lon,noon_lat,marker='o',color='red',label='Noon time')
color_bar=fig.colorbar(im)
color_bar.set_label('Solar Local Time (hr)')

plt.tight_layout(rect=[0, 0.0, 1.05, 0.95])
fig.autofmt_xdate(rotation=45)

plt.legend()

left  = 0.1  # the left side of the subplots of the figure
right = 0.9   # the right side of the subplots of the figure
bottom = 0.05   # the bottom of the subplots of the figure
top = 0.9     # the top of the subplots of the figure
wspace = 0.1#0.2   # the amount of width reserved for blank space between subplots
hspace = 0.8#0.2   # the amount of height reserved for white space between subplots

plt.subplots_adjust(left, bottom, right, top, wspace, hspace)



# Processing SABER Data -